In [1]:
from py2neo import Graph
import pandas as pd
import numpy as np
import os

In [2]:
graph = Graph("bolt://localhost:7687", auth=("superman", "pizzapep"))

In [3]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

# Making nodes and edges

In [4]:
def create_nodes(df):
    """Function to create the nodes from the csv files.

    Returns:
        str: Acknowledgement string
    """
    i=0
    for idx in df.index:
        id_ = df.loc[idx, 'Id']
        label = df.loc[idx, 'Label'].replace('\'', '')
        cypher = "MERGE (a:Person {id:" + "'" + \
            id_ + "'" + ",label:" + "'" + label + "'})"
        cypher = cypher + f'SET a.seed={i}'
        graph.run(cypher)
        i+=1
    return "Done creating nodes"

Note: for the function create_edges; I added + str(season)+ in order to have various edges depending on the interaction betwee the character in different seasons

In [5]:
def create_edges(df):
    """Function to create the edges from the csv files.

    Returns:
        str: Acknowledgement string
    """
    for idx in df.index:
        src = df.loc[idx, 'Source']
        tar = df.loc[idx, 'Target']
        weight = df.loc[idx, 'Weight']
        season = df.loc[idx, 'Season']
        cypher = "MATCH (src:Person {id:" + "'" + src + "'}),"
        cypher = cypher + " (tar:Person {id:" + "'" + tar + "'})"
        cypher = cypher + \
            "MERGE (src)-[r:INTERACTS_"+ str(season) +"]-(tar)"
        cypher = cypher + f"SET r.weight={weight} SET r.season={season}"
        graph.run(cypher)
    return "Done creating edges"

In [6]:
#Here we create an undirected edges
def create_graph():
    """Function to create the graph from the csv files.

    Returns:
        str: Acknowledgement string
    """
    for season in range(1,8):
        for type in ['nodes','edges']:
            base_url = "https://raw.githubusercontent.com/mathbeveridge/gameofthrones/master/data/"
            final_url = base_url+"got-s"+str(season)+"-"+type+".csv"
            df = pd.read_csv(final_url)
            if type == 'nodes':
                create_nodes(df)
            else:
                create_edges(df)
    return "Done creating graph"

In [7]:
create_graph()
#Note: season 8 is missing

'Done creating graph'

# Graph for season 1

## Louvain Modularity Algorithm

In [8]:
query = """
CALL gds.graph.project.cypher(
  'Got1',
  'MATCH (n:Person)-[r:INTERACTS_1]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_1]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got1,"MATCH (n:Person)-[r:INTERACTS_1]-(Person) RETURN id(n) AS id, n.seed AS seed",126,"MATCH (n:Person)-[r:INTERACTS_1]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",1098


In [9]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got1', { writeProperty: 'community_S1' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
126,1098,13369,582864,[13369 Bytes ... 569 KiB]


In [10]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got1', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Addam,117
Aegon,46
Aemon,125


In [11]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got1')
YIELD communityCount
"""
graph.run(query)

communityCount
6


In [12]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got1', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.4500051426504889,"[0.40857860458326284, 0.4500051426504889]"


In [13]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got1', { writeProperty: 'community_S1' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.4500051426504889,"[0.40857860458326284, 0.4500051426504889]"


## 5 basic centrality measures 

In [14]:
#Degree
query = """
CALL gds.degree.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
NED,57.0
TYRION,41.0
ROBERT,36.0


In [15]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got1',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
NED,1290.0
TYRION,709.0
CATELYN,584.0


In [16]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
NED,0.3150828603837995
ROBERT,0.2481904649529348
CERSEI,0.239263152443205


In [17]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got1',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
NED,5.710814289359831
TYRION,4.140412812879867
CATELYN,3.4397081230227586


In [18]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got1')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
NED,4700.84322629209
TYRION,2526.9252462619834
CATELYN,1833.4028207687145


# Graph for season 2

## Louvain Modularity Algorithm

In [19]:
query = """
CALL gds.graph.project.cypher(
  'Got2',
  'MATCH (n:Person)-[r:INTERACTS_2]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_2]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got2,"MATCH (n:Person)-[r:INTERACTS_2]-(Person) RETURN id(n) AS id, n.seed AS seed",129,"MATCH (n:Person)-[r:INTERACTS_2]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",972


In [20]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got2', { writeProperty: 'community_S2' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
129,972,13569,583456,[13569 Bytes ... 569 KiB]


In [21]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got2', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Aegon,123
Aerys,2
Alton,2


In [22]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got2')
YIELD communityCount
"""
graph.run(query)

communityCount
6


In [23]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got2', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.5675519483818524,"[0.5333155514911345, 0.5675519483818524]"


In [24]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got2', { writeProperty: 'community_S2' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.5675519483818524,"[0.5333155514911345, 0.5675519483818524]"


## 5 basic centrality measures 

In [25]:
#Degree
query = """
CALL gds.degree.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JOFFREY,36.0
TYRION,33.0
CERSEI,31.0


In [26]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got2',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
TYRION,960.0
CERSEI,549.0
ARYA,481.0


In [27]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JOFFREY,0.3133074261625616
CERSEI,0.28298193697415064
TYRION,0.27732429312701573


In [28]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got2',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
JOFFREY,3.6139514610118892
TYRION,3.4854517741397
ARYA,3.4014533093263664


In [29]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got2')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
ARYA,2455.3415562871237
TYRION,2293.4025669166035
ROBB,1902.715964088471


# Graph for season 3

## Louvain Modularity Algorithm

In [30]:
query = """
CALL gds.graph.project.cypher(
  'Got3',
  'MATCH (n:Person)-[r:INTERACTS_3]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_3]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got3,"MATCH (n:Person)-[r:INTERACTS_3]-(Person) RETURN id(n) AS id, n.seed AS seed",125,"MATCH (n:Person)-[r:INTERACTS_3]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",1014


In [31]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got3', { writeProperty: 'community_S3' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
125,1014,13305,582696,[13305 Bytes ... 569 KiB]


In [32]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got3', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Aegon,42
Aemon,15
Aerys,33


In [33]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got3')
YIELD communityCount
"""
graph.run(query)

communityCount
6


In [34]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got3', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.6260635131823116,"[0.6018171632645916, 0.6260635131823116]"


In [35]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got3', { writeProperty: 'community_S3' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
6,0.6260635131823116,"[0.6018171632645916, 0.6260635131823116]"


## 5 basic centrality measures 

In [36]:
#Degree
query = """
CALL gds.degree.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
ROBB,31.0
TYWIN,29.0
TYRION,25.0


In [37]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got3',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
TYRION,653.0
ROBB,375.0
TYWIN,366.0


In [38]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
TYWIN,0.294075789383587
TYRION,0.2683662004062233
JOFFREY,0.24983902616879472


In [39]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got3',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
ROBB,3.1307713054929844
TYWIN,2.874792630046823
TYRION,2.5522894149675976


In [40]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got3')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
ROBB,3609.6328036171367
NED,2651.602089578363
ROBERT,2453.263779280231


# Graph for season 4

## Louvain Modularity Algorithm

In [41]:
query = """
CALL gds.graph.project.cypher(
  'Got4',
  'MATCH (n:Person)-[r:INTERACTS_4]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_4]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got4,"MATCH (n:Person)-[r:INTERACTS_4]-(Person) RETURN id(n) AS id, n.seed AS seed",177,"MATCH (n:Person)-[r:INTERACTS_4]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",1410


In [42]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got4', { writeProperty: 'community_S4' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
177,1410,16641,592480,[16 KiB ... 578 KiB]


In [43]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got4', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Adrack Humble,86
Aemon,120
Aerys,147


In [44]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got4')
YIELD communityCount
"""
graph.run(query)

communityCount
8


In [45]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got4', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
8,0.6019938634877522,"[0.5523786529852623, 0.6019938634877522]"


In [46]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got4', { writeProperty: 'community_S4' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
8,0.6019938634877522,"[0.5523786529852623, 0.6019938634877522]"


## 5 basic centrality measures 

In [47]:
#Degree
query = """
CALL gds.degree.stream('Got4')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JOFFREY,41.0
TYRION,39.0
CERSEI,34.0


In [48]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got4',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
TYRION,890.0
CERSEI,632.0
JON,612.0


In [49]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got4')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JOFFREY,0.28749890520136145
TYRION,0.28114537272332196
CERSEI,0.26572628767163664


In [50]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got4',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
JOFFREY,3.995032308108388
TYRION,3.79855353319621
ARYA,3.337615317598997


In [51]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got4')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JOFFREY,6169.187204485663
NED,6077.094818619815
STANNIS,5446.0343108831285


# Graph for season 5

## Louvain Modularity Algorithm

In [52]:
query = """
CALL gds.graph.project.cypher(
  'Got5',
  'MATCH (n:Person)-[r:INTERACTS_5]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_5]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got5,"MATCH (n:Person)-[r:INTERACTS_5]-(Person) RETURN id(n) AS id, n.seed AS seed",124,"MATCH (n:Person)-[r:INTERACTS_5]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",866


In [53]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got5', { writeProperty: 'community_S5' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
124,866,13241,582456,[13241 Bytes ... 568 KiB]


In [54]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got5', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Aegon,18
Aemon,18
Aerys,101


In [55]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got5')
YIELD communityCount
"""
graph.run(query)

communityCount
5


In [56]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got5', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
5,0.664294971971689,"[0.6190789859671767, 0.664294971971689]"


In [57]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got5', { writeProperty: 'community_S5' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
5,0.664294971971689,"[0.6190789859671767, 0.664294971971689]"


## 5 basic centrality measures 

In [58]:
#Degree
query = """
CALL gds.degree.stream('Got5')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
CERSEI,31.0
LITTLEFINGER,27.0
JON,27.0


In [59]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got5',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
CERSEI,618.0
JON,616.0
TYRION,480.0


In [60]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got5')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
STANNIS,0.33835270473792883
JON,0.31716067325588637
SAM,0.283804814609472


In [61]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got5',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
JON,3.6049676053820563
CERSEI,3.604538477369172
LITTLEFINGER,3.5385256890743797


In [62]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got5')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
STANNIS,5012.7852459276455
CERSEI,3964.7548918141197
LITTLEFINGER,2971.9304254016747


# Graph for season 6

## Louvain Modularity Algorithm

In [63]:
query = """
CALL gds.graph.project.cypher(
  'Got6',
  'MATCH (n:Person)-[r:INTERACTS_6]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_6]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got6,"MATCH (n:Person)-[r:INTERACTS_6]-(Person) RETURN id(n) AS id, n.seed AS seed",148,"MATCH (n:Person)-[r:INTERACTS_6]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",1154


In [64]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got6', { writeProperty: 'community_S6' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
148,1154,14785,586976,[14785 Bytes ... 573 KiB]


In [65]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got6', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Aemon,34
Aeron,16
Aerys,11


In [66]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got6')
YIELD communityCount
"""
graph.run(query)

communityCount
9


In [67]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got6', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
9,0.6540944165272475,"[0.6090893253516515, 0.6540944165272475]"


In [68]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got6', { writeProperty: 'community_S6' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
9,0.6540944165272475,"[0.6090893253516515, 0.6540944165272475]"


## 5 basic centrality measures 

In [69]:
#Degree
query = """
CALL gds.degree.stream('Got6')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
SANSA,41.0
JON,33.0
CERSEI,29.0


In [70]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got6',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JON,624.0
SANSA,551.0
TYRION,442.0


In [71]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got6')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
SANSA,0.3519753949236122
JON,0.27646633533835374
DAVOS,0.23405853477802047


In [72]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got6',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
SANSA,3.894475250679934
JON,3.2502096784271135
CERSEI,3.007200929871433


In [73]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got6')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
SANSA,5791.99128590698
JON,4161.187582627684
TYRION,3080.729990101282


# Graph for season 7

## Louvain Modularity Algorithm

In [74]:
query = """
CALL gds.graph.project.cypher(
  'Got7',
  'MATCH (n:Person)-[r:INTERACTS_7]-(Person) RETURN id(n) AS id, n.seed AS seed',
  'MATCH (n:Person)-[r:INTERACTS_7]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight')
YIELD
    graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels

"""
graph.run(query)

graph,nodeQuery,nodes,relationshipQuery,rels
Got7,"MATCH (n:Person)-[r:INTERACTS_7]-(Person) RETURN id(n) AS id, n.seed AS seed",85,"MATCH (n:Person)-[r:INTERACTS_7]-(p:Person) RETURN id(n) AS source, id(p) AS target, r.weight AS weight",890


In [75]:
#The following will estimate the memory requirements for running the algorithm:
query = """
CALL gds.louvain.write.estimate('Got7', { writeProperty: 'community_S7' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
85,890,10745,575176,[10745 Bytes ... 561 KiB]


In [76]:
#The following will run the algorithm and stream results:
query = """
CALL gds.louvain.stream('Got7', { seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label ASC
"""
graph.run(query)

label,communityId
Aegon,45
Aerys,45
Alys,67


In [77]:
#The following will run the algorithm and returns the result in form of statistical and measurement values:
query = """
CALL gds.louvain.stats('Got7')
YIELD communityCount
"""
graph.run(query)

communityCount
4


In [78]:
#The following will run the algorithm and store the results in myGraph:
query = """
CALL gds.louvain.mutate('Got7', { mutateProperty: 'communityId' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
4,0.32572149981063003,"[0.3046484029794218, 0.32572149981063003]"


In [79]:
#The following run the algorithm, and write back results:
query = """
CALL gds.louvain.write('Got7', { writeProperty: 'community_S7' })
YIELD communityCount, modularity, modularities
"""
graph.run(query)

communityCount,modularity,modularities
4,0.32572149981063003,"[0.3046484029794218, 0.32572149981063003]"


## 5 basic centrality measures 

In [80]:
#Degree
query = """
CALL gds.degree.stream('Got7')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JON,46.0
TYRION,35.0
DAENERYS,35.0


In [81]:
#Weighted Degree
query = """
CALL gds.degree.stream(
   'Got7',
   { relationshipWeightProperty: 'weight' }
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JON,982.0
DAENERYS,735.0
TYRION,703.0


In [82]:
#Eigenvector
query = """
CALL gds.eigenvector.stream('Got7')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JON,0.2855926337551967
TYRION,0.2581673486956719
DAENERYS,0.2465015323052332


In [83]:
#Pagerank
query = """
CALL gds.pageRank.stream('Got7',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

page,score
JON,3.9485790707630444
DAENERYS,2.9040361981717644
TYRION,2.712399180043202


In [84]:
#Betweenness
query = """
CALL gds.betweenness.stream('Got7')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, score
ORDER BY score DESC
LIMIT 5
"""
graph.run(query)

name,score
JON,1761.0866789656602
SAM,851.8978738896036
DAENERYS,763.3372373706223


# Still we need to do:
- Figure out why season 8 is not in our graph
- Make analysis for season 8
- Conduct graph embeddings
- Think about predictions